## Week 10: Day 3 – Deeper Example

### Deep Neural Network For MNIST Classification

we will apply the knowledge from the lectures in this section to write a deep neural network.The problem we've chosen is referred to as the "Hello World" of deep learning because for most  students if the first deep learning algorithm they use.

The dataset is called MNIST and refers to handwritten digit recognition.You can find more about  it on Yann LeCun's website(Director of AI Research,Facebook).

The dataset provides 70,000 images (28×28 pixels) of handwritten digit (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0,1,2,3,4,5,6,7,8,9).this is a classification problem with 10 classes.

Our goal would be build a neural network with 2 hidden layer

#### Import the relevant packages

In [14]:
# import the required packages
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Data

#### Load a dataset


In [15]:
#tfds.loads(name) - loads a data from tensorflow dataset
# tfds.loads(name,with_info,as_supervized) :- 
# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 
# (1) as supervised - True,loads the data in a 2-tuple structure(inputs,targets)
# (2) with_info - True,provides a tuple containing information about version,features, #samples of the dataset
mnist_dataset,mnist_info = tfds.load('mnist',with_info = True, as_supervised = True)

mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']
# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
# this is also where we make use of mnist_info (we don't have to count the observations)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# let's cast this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# let's also store the number of test samples in a dedicated variable (instead of using the mnist_info one)
num_test_samples = mnist_info.splits['train'].num_examples
# once more, we'd prefer an integer (rather than the default float)
num_test_samples = tf.cast(num_validation_samples, tf.int64)

# normally, we would like to scale our data in some way to make the result more numerically stable
# in this case we will simply prefer to have inputs between 0 and 1
# let's define a function called: scale, that will take an MNIST image and its label
def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255
    return image, label


In [16]:
# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_dataset = mnist_test.map(scale)

BUFFER_SIZE = 10000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them
# if BUFFER_SIZE=1 => no shuffling will actually happen
# if BUFFER_SIZE >= num samples => shuffling is uniform
# BUFFER_SIZE in between - a computational optimization to approximate uniform shuffling

# there is a shuffle method readily available and we just need to specify the buffer size
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# once we have scaled and shuffled the data, we can proceed to actually extracting the train and validation
# our validation data would be equal to 10% of the training set, which we've already calculated
# we use the .take() method to take that many samples
# finally, we create a batch with a batch size equal to the total number of validation samples
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# the train_data is everything else, so we skip as many samples as there are in the validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [17]:
# determine the batch size
BATCH_SIZE = 100

# we can also take advantage of the occasion to batch the train data
# this would be very helpful when we train, as we would be able to iterate over the different batches
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples) # Single batch, having size equal to number of validation samples

# batch the test data
test_dataset = test_dataset.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Model

#### Outline the model

In [18]:
input_size = 784 # One for each pixel of the 28 * 28 image
output_size = 10
# Use same hidden layer size for both hidden layers.
hidden_layer_size = 50 # Arbitrary chosen

# define how the model will look like
model = tf.keras.Sequential([
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # First hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])



#### Choose the optimizer and the loss function

In [19]:
# we define the optimizer we'd like to use, the loss function, and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#### Training

In [20]:
# determine the maximum number of epochs
NUM_EPOCHS = 5
# we fit the model, specifying the training data the total number of epochs and the validation data we just created ourselves in the format:
# (inputs,targets)
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 15s - loss: 0.4117 - accuracy: 0.8847 - val_loss: 0.2135 - val_accuracy: 0.9408
Epoch 2/5
540/540 - 1s - loss: 0.1886 - accuracy: 0.9449 - val_loss: 0.1815 - val_accuracy: 0.9462
Epoch 3/5
540/540 - 1s - loss: 0.1405 - accuracy: 0.9589 - val_loss: 0.1261 - val_accuracy: 0.9642
Epoch 4/5
540/540 - 2s - loss: 0.1137 - accuracy: 0.9656 - val_loss: 0.1149 - val_accuracy: 0.9672
Epoch 5/5
540/540 - 1s - loss: 0.0951 - accuracy: 0.9713 - val_loss: 0.0953 - val_accuracy: 0.9717


#### Test the model

In [22]:
test_loss, test_accuracy = model.evaluate(test_dataset)

2/2 [==============================] - 1s 39ms/step - loss: 0.1000 - accuracy: 0.9685


In [23]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 96.85%
